<a href="https://colab.research.google.com/github/cedamusk/final-year/blob/main/cookscript2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install obspy matplotlib tensorflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from obspy import UTCDateTime, Stream, Trace
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.filter import bandpass
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Bidirectional, SimpleRNN
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score


In [ ]:
def generate_synthetic_data(num_samples, sample_rate, event_duration, noise_level, num_events=1):
  time=np.arange(num_samples)/sample_rate
  background=np.random.normal(0, noise_level, num_samples)

  events=np.zeros(num_samples)
  event_starts=[]
  event_ends=[]

  for _ in range(num_events):
    event_start=np.random.randint(0, num_samples-int(event_duration*sample_rate))
    event_end=event_start+int(event_duration*sample_rate)
    event=np.sin(2*np.pi*5*(time[event_start:event_end]-time[event_start]))*np.exp(-(time[event_start:event_end]-time[event_start])/0.2)
    events[event_start:event_end]+=event
    event_starts.append(event_start)
    event_ends.append(event_end)

  data=background+events
  return data, events, event_starts, event_ends


In [ ]:
num_samples=50000
sample_rate=100
event_duration=2
noise_level=0.1
while True:
  synthetic_data, true_event, event_starts, event_ends=generate_synthetic_data(num_samples, sample_rate, event_duration, noise_level, num_events=5)

  true_events=np.zeros(len(synthetic_data))
  for start, end in zip(event_starts, event_ends):
    true_events[start:end]=1

  if len(np.unique(true_events))==2:
    break
  else:
    print("Generated data has only one class...Retrying...")


In [ ]:
start_time=UTCDateTime("2021-01-01T00:00:00")
trace=Trace(data=synthetic_data)
trace.stats.starttime=start_time
trace.stats.delta=1.0/sample_rate
trace.stats.channel='SHZ'
stream=Stream([trace])

In [ ]:
filtered_stream=stream.copy()
filtered_stream.filter('bandpass', freqmin=0.5, freqmax=20, corners=4, zerophase=True)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(np.arange(num_samples)/sample_rate, synthetic_data)
plt.title("Synthetic Seismic Data")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
plt.show()


In [ ]:
filtered_stream=stream.copy()
filtered_stream.filter('bandpass', freqmin=0.5, freqmax=20, corners=4, zerophase=True)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Synthetic Seismic Data")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
def sta_lta_detection(stream, sta_length, lta_length, threshold):
  data=stream[0].data
  sta_lta=classic_sta_lta(data, sta_length, lta_length)
  triggers=np.where(sta_lta>threshold)[0]
  return triggers, sta_lta

In [ ]:
sta_length=int(0.5*sample_rate)
lta_length=int(5*sample_rate)
threshold=3

triggers, sta_lta=sta_lta_detection(filtered_stream, sta_length, lta_length, threshold)


In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Seismic Data with Detected Events")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
for trigger in triggers:
  plt.axvline(x=trigger/sample_rate, color='r', linestyle='--')

plt.subplot(2, 1, 2)
plt.plot(np.arange(len(sta_lta))/sample_rate, sta_lta)
plt.axhline(y=threshold, color='r', linestyle='--')
plt.title("STA/LTA Ratio")
plt.tight_layout()
plt.show()

In [ ]:
window_size=100
step=10

In [ ]:
def create_windows(data, window_size, step):
  return np.array([data[i:i+window_size]for i in range(0, len(data)-window_size+1, step)])


In [ ]:
X=create_windows(filtered_stream[0].data, window_size, step)
y=np.zeros(len(X))

In [ ]:
for i, window_start in enumerate(range(0, len(filtered_stream[0].data)-window_size+1, step)):
  window_end=window_start+window_size
  if np.any(true_events[window_start:window_end]==1):
    y[i]=1

In [ ]:
for trigger in triggers:
  event_windows=np.where((np.arange(len(X))*step<=trigger)&(trigger<np.arange(len(X))*step+window_size))[0]
  y[event_windows]=1

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_mean=X_train.mean()
X_train_std=X_train.std()
X_train=(X_train-X_train_mean)/X_train_std
X_test=(X_test-X_train_mean)/X_train_std

In [ ]:
X_train=X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test=X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
model=Sequential([
    Bidirectional(LSTM(64, return_sequences=True, input_shape=(window_size, 1))),
    Bidirectional(LSTM(32, return_sequences=True)),
    GRU(16),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
loss, accuracy=model.evaluate(X_test, y_test)
print(f"Test accuracy:{accuracy:.4f}")

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
def rnn_detection(data, model, window_size, step):
  X=create_windows(data, window_size, step)
  X=(X-X_train_mean)/X_train_std
  X=X.reshape((X.shape[0], X.shape[1],1))
  predictions=model.predict(X)
  return predictions.flatten()

In [ ]:
rnn_predictions=rnn_detection(filtered_stream[0].data, model, window_size, step)

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(3,1,1)
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Seismic Data")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")

plt.subplot(3,1,2)
plt.plot(np.arange(len(sta_lta))/sample_rate, sta_lta)
plt.axhline(y=threshold, color='r', linestyle='--')
plt.title("STA/LTA Ratio")
plt.xlabel("Time(s)")
plt.ylabel("STA/LTA Ratio")

plt.subplot(3,1,3)
plt.plot(np.arange(len(rnn_predictions))*step/sample_rate, rnn_predictions)
plt.title("RNN Predictions")
plt.xlabel("Time(s)")
plt.ylabel("Probability")

plt.tight_layout()
plt.show()

In [ ]:
num_rnn_predictions=len(rnn_predictions)
sta_lta_trimmed=sta_lta[:num_rnn_predictions]
combined_detections=(sta_lta_trimmed>threshold)&(rnn_predictions>0.5)
combined_triggers=np.where(combined_detections)[0]

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data)
plt.title("Filtered Seismic Data with Combined Detections")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
for trgger in combined_triggers:
  plt.axvline(x=trigger/sample_rate, color='g', linestyle='--')
  plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(np.arange(num_samples)/sample_rate, synthetic_data)
plt.title("Synthetic Seismic Data")
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
for start, end in zip(event_starts, event_ends):
  plt.axvspan(start/sample_rate, end/sample_rate, color='r', alpha=0.5)
plt.show()

In [ ]:
event_starts_idx=[int(start*sample_rate)for start in event_starts]
event_ends_idx=[int(end*sample_rate)for end in event_ends]

true_events=np.zeros(len(filtered_stream[0].data))
for start, end in zip(event_starts_idx, event_ends_idx):
  start=max(0, start)
  end=min(len(true_events), end)
  true_events[start:end]=1

In [ ]:
if len(np.unique(true_events))!=2:
  print("ROC AUC cannot be calculated because ture_events contains only one class")
else:
  from sklearn.metrics import roc_auc_score
  auc_score=roc_auc_score(true_events, predicted_probabilities)
  print(f"ROC AUC:{roc_auc_score(true_events, rnn_predictions):.4f}")

In [ ]:
sta_lta_predictions=np.zeros(len(filtered_stream[0].data))
sta_lta_predictions[triggers]=1

In [ ]:
rnn_predictions_full=np.zeros(len(filtered_stream[0].data))
rnn_predictions_full[np.arange(len(rnn_predictions))*step]=(rnn_predictions>0.5).astype(int)

In [ ]:
combined_predictions=np.zeros(len(filtered_stream[0].data))
combined_predictions[combined_triggers]=1

In [ ]:
def print_metrics(y_true, y_pred, y_pred_proba, method_name):
  if len(np.unique(y_true))<2:
    print(f"{method_name}Metrics")
    print("Cannot calculate metrics. All true labels are the same")
    print()
    return

  with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

  precision=precision_score(y_true, y_pred)
  recall=recall_score(y_true, y_pred)
  f1=f1_score(y_true, y_pred)
  auc=roc_auc_score(y_true, y_pred_proba)
  print(f"{method_name} Metrics:")
  print(f"Precision:{precision:.4f}")
  print(f"Recall:{recall:.4f}")
  print(f"f1 Score:{f1:.4f}")

  try:
    auc=roc_auc_score(y_true, y_pred_proba)
    print(f"AUC:{auc:.4f}")
  except ValueError:
    print("AUC cannot be calculated(requires both classes)")
  print()

In [ ]:
sta_lta_binary=(sta_lta>threshold).astype(int)
sta_lta_proba=sta_lta/np.max(sta_lta)

In [ ]:
combined_binary=np.logical_or(sta_lta_binary, rnn_predictions_full).astype(int)
if len(sta_lta_proba)<len(rnn_predictions):
  sta_lta_proba=np.pad(sta_lta_proba, (0, len(rnn_predictions)-len(sta_lta_proba)), 'constant')
else:
  rnn_predictions=np.pad(rnn_predictions, (0, len(sta_lta_proba)-len(rnn_predictions)), 'constant')
combined_proba=np.maximum(sta_lta_proba, rnn_predictions)

In [ ]:
print_metrics(true_events, sta_lta_binary, sta_lta_proba, "STA/LTA")
print_metrics(true_events, rnn_predictions_full, rnn_predictions, "RNN")
print_metrics(true_events, combined_predictions, combined_proba, "Combined STA/LTA and RNN")

In [ ]:
from sklearn.metrics import roc_curve
plt.figure(figsize=(10,8))
fpr,tpr,_=roc_curve(true_events, sta_lta/np.max(sta_lta))
plt.plot(fpr, tpr, label=f'STA/LTA(AUC={roc_auc_score(true_events, sta_lta/np.max(sta_lta)):.2f})')

fpr, tpr, _=roc_curve(true_events, rnn_predictions)
plt.plot(fpr,tpr, label=f'RNN(AUC={roc_auc_score(true_events, rnn_predictions):.2f})')

fpr, tpr, _=roc_curve(true_events, np.maximum(sta_lta/np.max(sta_lta), rnn_predictions))
plt.plot(fpr, tpr, label=f'Combined (AUC={roc_auc_score(true_events, np.maximum(sta_lta/np.max(sta_lta), rnn_predictions)):.2f})')

plt.plot([0,1], [0,1], linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(np.arange(num_samples)/sample_rate, filtered_stream[0].data, label='Filtered Data' )
plt.plot(np.arange(num_samples)/sample_rate, true_events*np.max(filtered_stream[0].data), 'g', label='True Event')
plt.plot(np.arange(num_samples)/sample_rate, combined_predictions*np.max(filtered_stream[0].data),'r--', label='Combined Detection')
plt.title('True Event vs Combined Detection')
plt.xlabel("Time(s)")
plt.ylabel("Amplitude")
plt.legend()
plt.show()